# Apply Random Forest

In [1]:
library(grid)
library(rpart)
library(rpart.plot)
library(partykit)
library(lattice)
library(ggplot2)
library(caret)
library(e1071)
library(dplyr)
library(randomForest)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin



In [2]:
set.seed(1)

In [3]:
valuables <- c("RoadType","CurveAverage","Speed", "Curve100", "Curve150", "MaxSpeed","RiskFactor","Curve","DistSignal","Pitch","AheadDistance","AverageVelocity","TimeToCollision","AccelerationSpeed", "Engine", "SteeringAngle", "TimeHeadway", "Jerk", "LaneCount", "DiffAvgSpeed", "EmptinessOfRoad", "RoadFactor")

In [4]:
printf <- function(...) cat(sprintf(...))

In [5]:
showMatrix <- function(test, predictedFlags) {
    predictedRed <- test[predictedFlags == "Red", ]
    predictedBlue <- test[predictedFlags == "Blue", ]
    printf("Predict Red\n")
    print(predictedRed %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
    printf("Predict Blue\n")
    print(predictedBlue %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
}

# Predict Red

In [6]:
train <-  read.csv("../data/middle/sp5.csv", stringsAsFactors=FALSE)

In [7]:
test <- read.csv("../data/middle/sp6.csv", stringsAsFactors=FALSE)

In [8]:
train %>% group_by(flag) %>% summarize(count=n())

flag,count
BlueA,77
BlueB,372
RedA,401
RedB,192


In [9]:
test %>% group_by(flag) %>% summarize(count=n())

flag,count
BlueA,57
BlueB,148
RedA,181
RedB,90


In [10]:
test$AccelOrBrake[test$flag == "RedA"] <- "Accel"
test$AccelOrBrake[test$flag == "RedB"] <- "Brake"
test$AccelOrBrake[test$flag == "BlueA"] <- "Accel"
test$AccelOrBrake[test$flag == "BlueB"] <- "Brake"

In [11]:
train$flag[train$flag == "RedA"] <- "Red"
train$flag[train$flag == "RedB"] <- "Red"
train$flag[train$flag == "BlueA"] <- "Blue"
train$flag[train$flag == "BlueB"] <- "Blue"
train$flag <- as.factor(train$flag)

In [12]:
test$flag[test$flag == "RedA"] <- "Red"
test$flag[test$flag == "RedB"] <- "Red"
test$flag[test$flag == "BlueA"] <- "Blue"
test$flag[test$flag == "BlueB"] <- "Blue"
test$flag <- as.factor(test$flag)

In [13]:
train <- train[, c(valuables, "flag")]
test <- test[, c(valuables, "AccelOrBrake", "flag")]

In [14]:
set.seed(1)

In [15]:
fit <- randomForest(flag ~ ., data=train, mtry=3, ntree=500)
p <- predict(fit, newdata=test)

showMatrix(test, p)

nerr <- sum(p != test$flag)
printf("Err: %f", nerr / nrow(test))

Predict Red
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel    30
2   Blue        Brake    52
3    Red        Accel    99
4    Red        Brake    50
Predict Blue
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel    27
2   Blue        Brake    96
3    Red        Accel    82
4    Red        Brake    40
Err: 0.428571

In [16]:
for (i in c(0, 2, 3, 4, 5, 6, 7))  {
    printf("RoadType: %d\n", i)
    X <- train[train$RoadType == i, ]
    y <- test[test$RoadType == i, ]
    fit <- randomForest(flag ~ ., data=X, mtry=3, ntree=300)
    p <- predict(fit, newdata=y)

    showMatrix(test, p)

    nerr <- sum(p != y$flag)
    printf("Err: %f\n\n",  nerr / nrow(y))
}

RoadType: 0
Predict Red
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel    51
2   Blue        Brake   128
3    Red        Accel   158
4    Red        Brake    80
Predict Blue
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel     6
2   Blue        Brake    20
3    Red        Accel    23
4    Red        Brake    10
Err: 0.395833

RoadType: 2
Predict Red
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel    47
2   Blue        Brake   125
3    Red        Accel   154
4    Red        Brake    74
Predict Blue
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel    10
2   Blue        Brake    23
3    Red        Accel    27
4    Red        Brake    16
Err: 0.315789

RoadType: 3
Predict 

In [17]:
y %>% group_by(flag) %>% summarize(count=n())

flag,count
Blue,1
Red,2


In [22]:
test[p == "Blue", ]

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”

RoadType,CurveAverage,Speed,Curve100,Curve150,MaxSpeed,RiskFactor,Curve,DistSignal,Pitch,⋯,Engine,SteeringAngle,TimeHeadway,Jerk,LaneCount,DiffAvgSpeed,EmptinessOfRoad,RoadFactor,AccelOrBrake,flag
